In [1]:
import numpy as np
import pandas as pd

In [2]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
tokenizer = RegexpTokenizer(r'\w+')
stemmer = SnowballStemmer('english')
st = WordNetLemmatizer()

## Function

In [3]:
def generateDic(item):
    text = " ".join([st.lemmatize(stemmer.stem(word),'v') 
                     for word in tokenizer.tokenize(item.lower()) if word not in stopwords.words('english')])
    dic = dict()
    for w in text.split():
        if w in dic.keys():
            dic[w] = dic[w]+1
        else:
            dic[w] = 1
    return dic

# Attributes

In [4]:
#attributes=pd.read_csv('attributes.csv')

In [5]:
#attributes.head(2)

In [6]:
#attributes['att_Dict']=attributes.value.map(lambda x: generateDic(x))

In [7]:
#attributes.head()

# Description

In [8]:
#product_descriptions=pd.read_csv('product_descriptions.csv')

In [9]:
#product_descriptions.head()

In [10]:
#product_descriptions.index[1:10]

In [11]:
#product_descriptions['p_d_Dict']=product_descriptions.product_description[0:1000].map(lambda x: generateDic(x))

In [12]:
#product_descriptions.head()

# Train

In [13]:
train=pd.read_csv('train_clean.csv')

In [14]:
train.shape

(74067, 9)

In [15]:
train.head(2)

,id,product_uid,product_title,search_term,product_description,relevance,productDict,searchDict,desDict
0,2,100001,Simpson Strong-Tie 12-Gauge Angle,angle bracket,"Not only do angles make joints stronger, they ...",3.0,"{u'12': 1, u'angl': 1, u'gaug': 1, u'tie': 1, ...","{u'angl': 1, u'bracket': 1}","{u'repair': 1, u'outdoor': 1, u'zmax': 1, u'ho..."
1,3,100001,Simpson Strong-Tie 12-Gauge Angle,l bracket,"Not only do angles make joints stronger, they ...",2.5,"{u'12': 1, u'angl': 1, u'gaug': 1, u'tie': 1, ...","{u'bracket': 1, u'l': 1}","{u'repair': 1, u'outdoor': 1, u'zmax': 1, u'ho..."


  #    

##   

# TF-IDF

## Common Vocabulary

In [69]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(analyzer='word', ngram_range=(1,2), min_df = 0, stop_words = 'english')

In [17]:
tfidf_matrix=tf.fit_transform(train.product_description)

In [18]:
feature_names = tf.get_feature_names()

In [19]:
dense = tfidf_matrix.todense()

In [20]:
dense.shape, dense[0].shape, dense[0].shape

((74067, 1373750), (1, 1373750), (1, 1373750))

In [21]:
((dense[1]==dense[2])==True).sum()

1373405

In [22]:
episode=dense[100].tolist()[0]

In [23]:
phrase_scores = [pair for pair in zip(range(0, len(episode)), episode) if pair[1] > 0]

In [24]:
sorted(phrase_scores, key=lambda t: t[1] * -1)[:3]

[(938217, 0.3111114557886197),
 (122660, 0.23748638962660262),
 (560794, 0.21870461599741992)]

In [25]:
feature_names[122660]

u'automatically power'

In [26]:
#sorted_phrase_scores = sorted(phrase_scores, key=lambda t: t[1] * -1)
#for phrase, score in [(feature_names[word_id], score) for (word_id, score) in sorted_phrase_scores]:
#    print ('{0: <30} {1}'.format(phrase, score))

In [27]:
train['tfidf']=0
for i in range(len(dense)):
    aadict = {}
    episode=dense[i].tolist()[0]
    phrase_scores = [pair for pair in zip(range(0, len(episode)), episode) if pair[1] > 0]
    sorted_phrase_scores = sorted(phrase_scores, key=lambda t: t[1] * -1)
    for phrase, score in [(feature_names[word_id], score) for (word_id, score) in sorted_phrase_scores]:
        aadict[phrase] = score
        train['tfidf'][i] = [aadict]
    #train['tfidf']=train['product_description'].map(lambda x: common_vocabulary(x))
    if i>2:
        break

/Users/pengfeiwang/Documents/Anaconda2/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/pengfeiwang/Documents/Anaconda2/anaconda/lib/python2.7/site-packages/pandas/core/indexing.py:117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [61]:
sorted(train.tfidf[1][0].items())

[(u'10d', 0.08256010878710593),
 (u'10d common', 0.08455465492225152),
 (u'12', 0.03614672934380944),
 (u'12 gauge', 0.07343059894081727),
 (u'90', 0.047772390747461056),
 (u'90 connections', 0.10689579463957886),
 (u'alonehelp', 0.10689579463957886),
 (u'alonehelp ensure', 0.10689579463957886),
 (u'angled', 0.06309839947870056),
 (u'angled nailing', 0.09688162305724855),
 (u'angles', 0.13872805675259733),
 (u'angles make', 0.10144374201564983),
 (u'angles various', 0.10144374201564983),
 (u'bent', 0.07791394985884452),
 (u'bent skewed', 0.10144374201564983),
 (u'coated', 0.04321898503872383),
 (u'coated connectors', 0.10144374201564983),
 (u'common', 0.04877288023908281),
 (u'common nails', 0.07745317236071679),
 (u'connection', 0.04907384097680511),
 (u'connection needed', 0.10075492102245356),
 (u'connections', 0.056150408265782714),
 (u'connections home', 0.10689579463957886),
 (u'connector', 0.05776846062526235),
 (u'connector various', 0.10689579463957886),
 (u'connectors', 0.060

In [66]:
#train.tfidf[1]
train.product_description[1] 

'Not only do angles make joints stronger, they also provide more consistent, straight corners. Simpson Strong-Tie offers a wide variety of angles in various sizes and thicknesses to handle light-duty jobs or projects where a structural connection is needed. Some can be bent (skewed) to match the project. For outdoor projects or those where moisture is present, use our ZMAX zinc-coated connectors, which provide extra resistance against corrosion (look for a "Z" at the end of the model number).Versatile connector for various 90 connections and home repair projectsStronger than angled nailing or screw fastening aloneHelp ensure joints are consistently straight and strongDimensions: 3 in. x 3 in. x 1-1/2 in.Made from 12-Gauge steelGalvanized for extra corrosion resistanceInstall with 10d common nails or #9 x 1-1/2 in. Strong-Drive SD screws'

## Individual Vocabulary 

In [29]:
# combine the three dictionary tpgether and than do the tfidf

In [30]:
new=train.groupby('product_uid').agg(lambda x: x.tolist())

In [31]:
new.head(2)

,id,product_title,search_term,product_description,relevance,productDict,searchDict,desDict,tfidf
product_uid,,,,,,,,,
100001,"[2, 3]","[Simpson Strong-Tie 12-Gauge Angle, Simpson St...","[angle bracket, l bracket]","[Not only do angles make joints stronger, they...","[3.0, 2.5]","[{u'12': 1, u'angl': 1, u'gaug': 1, u'tie': 1,...","[{u'angl': 1, u'bracket': 1}, {u'bracket': 1, ...","[{u'repair': 1, u'outdoor': 1, u'zmax': 1, u'h...","[[{u'outdoor': 0.0341927119211, u'zmax': 0.100..."
100002,[9],[BEHR Premium Textured DeckOver 1-gal. #SC-141...,[deck over],[BEHR Premium Textured DECKOVER is an innovati...,[3.0],"[{u'tugboat': 1, u'deckov': 1, u'premium': 1, ...",[{u'deck': 1}],"[{u'galloncr': 1, u'sidewalks100': 1, u'compos...","[[{u'sidewalks100': 0.0814258189897, u'applica..."


In [32]:
train.shape,new.shape

((74067, 10), (54667, 9))

In [33]:
new.search_term.values

array([['angle bracket', 'l bracket'], ['deck over'],
       ['rain shower head', 'shower only faucet'], ...,
       ['schlage lock siena half dummy knob with'], ['zen garden  decor'],
       ['fine sheer curtain 63 inches']], dtype=object)

In [34]:
#create dict for sear-term individually
str_total=""
for i in range(len(new.search_term)):
    str1 = ''.join(str(e) for e in new.search_term.iloc[i])
    str_total=str_total+str1

In [35]:
tf = TfidfVectorizer(analyzer='word', ngram_range=(1,2), min_df = 0, stop_words = 'english')
tfidf_matrix_new_s_t=tf.fit_transform(pd.Series(str_total.split()))

In [36]:
tfidf_matrix_new_s_t

<159927x64142 sparse matrix of type '<type 'numpy.float64'>'
	with 151107 stored elements in Compressed Sparse Row format>

In [37]:
feature_names_new_s_t = tf.get_feature_names()

In [38]:
dense_search_term_indi = tfidf_matrix.todense()

In [39]:
dense_search_term_indi.shape, dense_search_term_indi[0].shape,dense_search_term_indi[0].shape

((74067, 1373750), (1, 1373750), (1, 1373750))

In [40]:
episode_search_term_indi=dense[100].tolist()[0]

In [41]:
phrase_scores_search_term_indi = [pair for pair in zip(range(0, len(episode_search_term_indi)), episode_search_term_indi) if pair[1] > 0]

In [42]:
sorted(phrase_scores_search_term_indi, key=lambda t: t[1] * -1)[:5]

[(938217, 0.3111114557886197),
 (122660, 0.23748638962660262),
 (560794, 0.21870461599741992),
 (623338, 0.20333274938488016),
 (1270208, 0.1963557866357005)]

In [43]:
feature_names_new_s_t[1270208]

IndexError: list index out of range

In [ ]:
def df_itf(item):
    tf = TfidfVectorizer(analyzer='word', ngram_range=(1,2), min_df = 0, stop_words = 'english')  #at most gram-2
    tfidf_matrix=tf.fit_transform(item)   #item should be pd.Series
    feature_names = tf.get_feature_names()
    dense = tfidf_matrix.todense()
    episode=dense[100].tolist()[0]  # take the row 100 as an example. crush
    phrase_scores = [pair for pair in zip(range(0, len(episode)), episode) if pair[1] > 0]
    train['tfidf']=0
    for i in range(len(dense)):
        aadict = {}
        episode=dense[i].tolist()[0]
        phrase_scores = [pair for pair in zip(range(0, len(episode)), episode) if pair[1] > 0]
        sorted_phrase_scores = sorted(phrase_scores, key=lambda t: t[1] * -1)
        for phrase, score in [(feature_names[word_id], score) for (word_id, score) in sorted_phrase_scores]:
            aadict[phrase] = score
            train['tfidf'][i] = [aadict]
            #if i>2:
            #    break

In [ ]:
aadict.items()[1:10]

### SDT

In [52]:
import numpy as np
from sklearn.decomposition import TruncatedSVD

In [59]:
TruncatedSVD(aadict)

TruncatedSVD(algorithm='randomized',
       n_components={u'code': 0.060885306620496724, u'help': 0.04039450573496891, u'rough kit': 0.09946039214734335, u'minimalistic': 0.11270632493110236, u'sleek': 0.05585122738990725, u'kit r10000': 0.11270632493110236, u'proposition': 0.03416816727205851, u'nbsp': 0.034124196795851824, u'rough': 0.06472...tion': 0.03416910447801144, u'multichoice': 0.1068243065336903, u'vero single': 0.12194447120620736},
       n_iter=5, random_state=None, tol=0.0)

In [60]:
TruncatedSVD?